# Article...

## Setup

In [1]:
!git clone https://github.com/NesyaLab/qaoa/

Cloning into 'qaoa'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 69 (delta 27), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (69/69), 147.95 KiB | 2.11 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [2]:
import os
os.chdir("/Users/main/Desktop/qaoa")

In [3]:
#!pip install -r requirements.txt

In [4]:
from config import *

In [5]:
from classes import Problems as P
from classes import Qaoa as Q
from functions import qaoa_utilities as qaoa_utils
from functions import maxcut_utilities as mcut_utils
from functions import qaoa_optimizers as optims
from functions import symmetry_utilities as sym_utils

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import sys

## Get data

In [7]:
graphs = []
for i in range(num_graphs):
    graph_path =  "/graph_"+str(i)+".nx"
    with open("data/graph_"+str(i)+".nx", 'rb') as f:
        g = pickle.load(f)
    graphs.append(g)

## Results

In [8]:
!mkdir results
results_path = "/results"

### Brute force

In [9]:
bfs = [0]*len(graphs)
idx = 0
for graph in graphs:
    bfs[idx] = mcut_utils.compute_max_cut_exactly(graph)
    idx += 1

with open("results/brute_force_solution", 'wb') as file:
    pickle.dump(bfs, file)

### Qaoa vanilla

In [10]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_vanilla_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            qaoa = Q.Qaoa(p=layer, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with one "shadow" node

In [11]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_1_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with two "shadow" nodes

In [12]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_2_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            G_.add_node(N+1)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with one pendent edge

In [13]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_3_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            u = list(G_.nodes())[N]
            v = list(G_.nodes())[random.randint(0,N-1)]
            G_.add_edge(u,v)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, _ = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            qaoa = Q.Qaoa(p=layer, G=graphs[idx], betas=x[:layer], gammas=x[layer:], mixer=mixer, seed=seed_, verbose=verbose)
            _, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA reduced

In [14]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_reduced_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            G_ = graphs[idx].get_graph().copy()
            G_prime = P.Problems(p_type="custom", G=G_)
        opt = [0]
        for edge in G_prime.get_edges():
            G_reduced = G_prime.get_graph().copy()
            G_reduced.remove_edges_from([edge])
            reduced_graph_instance = P.Problems(p_type="custom", G=G_reduced)
            qaoa = Q.Qaoa(p=layer, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            opt.append(AR)
        with open("results/" + name, 'wb') as file:
            pickle.dump((x,f,max(opt)), file)

### Spectrum

In [15]:
# CALCULATING SPECTRAL RADIUS METRIC FOR EACH GRAPH

idx = 0
metric = {}
spectral_radii = []
avg_degrees = []
for graph in graphs:
    # The max eigenvalue is real
    rho = float(max(graph.get_adjacency_spectrum()))
    D = np.diag(np.sum(graph.get_adjacency_matrix(), axis=1))
    spectral_radii.append(rho)
    avg_degrees.append(int(np.sum(D)/D.shape[0]))
    metric[idx] = [avg_degrees[idx] / spectral_radii[idx]]
    idx += 1
with open("results/spectral_radii", 'wb') as file:
    pickle.dump(spectral_radii, file)
with open("results/spectral_metric_unperturbed", 'wb') as file:
    pickle.dump(metric, file)

/var/folders/fn/b9jm0kpj3k51rtzysg66fpnr0000gn/T/ipykernel_68923/3833129541.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  rho = float(max(graph.get_adjacency_spectrum()))


In [16]:
# CALCULATING SPECTRAL RADIUS METRIC FOR EACH GRAPH PERTURBATION

# One shadow node
idx = 0
metric_shadow1 = {}
spectral_radii_shadow1 = []
avg_degrees_shadow1 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_shadow1.append(rho)
    avg_degrees_shadow1.append(int(np.sum(D)/D.shape[0]))
    metric_shadow1[idx] = [avg_degrees_shadow1[idx] / spectral_radii_shadow1[idx]]
    idx += 1
with open("results/spectral_radii_shadow1", 'wb') as file:
    pickle.dump(spectral_radii_shadow1, file)
with open("results/spectral_metric_shadow1", 'wb') as file:
    pickle.dump(metric_shadow1, file)

# Two shadow nodes
idx = 0
metric_shadow2 = {}
spectral_radii_shadow2 = []
avg_degrees_shadow2 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    G.add_node(N+1)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_shadow2.append(rho)
    avg_degrees_shadow2.append(int(np.sum(D)/D.shape[0]))
    metric_shadow2[idx] = [avg_degrees_shadow2[idx] / spectral_radii_shadow2[idx]]
    idx += 1
with open("results/spectral_radii_shadow2", 'wb') as file:
    pickle.dump(spectral_radii_shadow2, file)
with open("results/spectral_metric_shadow2", 'wb') as file:
    pickle.dump(metric_shadow2, file)


# One pendent edge
idx = 0
metric_pendent = {}
spectral_radii_pendent = []
avg_degrees_pendent = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    u = list(G.nodes())[N]
    v = list(G.nodes())[random.randint(0,N-2)]
    G.add_edge(u,v)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_pendent.append(rho)
    avg_degrees_pendent.append(int(np.sum(D)/D.shape[0]))
    metric_pendent[idx] = [avg_degrees_pendent[idx] / spectral_radii_pendent[idx]]
    idx += 1
with open("results/spectral_radii_pendent", 'wb') as file:
    pickle.dump(spectral_radii_pendent, file)
with open("results/spectral_metric_pendent", 'wb') as file:
    pickle.dump(metric_pendent, file)

# One deleted edge
idx = 0
metric_deleted = {}
spectral_radii_deleted = {}
avg_degrees_deleted = {}
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G_ = P.Problems(p_type="custom", G=G)
    metric_deleted_vec = []
    avg_degrees_deleted_vec = []
    spectral_radii_deleted_vec = []
    for edge in G_.get_edges():
        G_reduced = G_.get_graph().copy()
        G_reduced.remove_edges_from([edge])
        instance = P.Problems(p_type="custom", G=G_reduced)
        rho = float(max(instance.get_adjacency_spectrum()))
        D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
        spectral_radii_deleted_vec.append(rho)
        avg_degrees_deleted_vec.append(int(np.sum(D)/D.shape[0]))
    spectral_radii_deleted[idx] = max(spectral_radii_deleted_vec)
    avg_degrees_deleted[idx] = max(avg_degrees_deleted_vec)
    metric_deleted[idx] = [avg_degrees_deleted[idx] / spectral_radii_deleted[idx]]
    idx += 1
with open("results/spectral_radii_deleted", 'wb') as file:
    pickle.dump(spectral_radii_deleted, file)
with open("results/spectral_metric_deleted", 'wb') as file:
    pickle.dump(metric_deleted, file)

/var/folders/fn/b9jm0kpj3k51rtzysg66fpnr0000gn/T/ipykernel_68923/3912479800.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  rho = float(max(instance.get_adjacency_spectrum()))
/var/folders/fn/b9jm0kpj3k51rtzysg66fpnr0000gn/T/ipykernel_68923/3912479800.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
  rho = float(max(instance.get_adjacency_spectrum()))
/var/folders/fn/b9jm0kpj3k51rtzysg66fpnr0000gn/T/ipykernel_68923/3912479800.py:60: ComplexWarning: Casting complex values to real discards the imaginary part
  rho = float(max(instance.get_adjacency_spectrum()))
/var/folders/fn/b9jm0kpj3k51rtzysg66fpnr0000gn/T/ipykernel_68923/3912479800.py:87: ComplexWarning: Casting complex values to real discards the imaginary part
  rho = float(max(instance.get_adjacency_spectrum()))


In [17]:
# CHECK

print(metric, "\n")
print(metric_shadow1, "\n")
print(metric_shadow2, "\n")
print(metric_pendent, "\n")
print(metric_deleted)

{0: [1.0000000000000002], 1: [1.0000000000000002], 2: [1.0000000000000004], 3: [1.0000000000000002], 4: [0.7071067811865475], 5: [0.7151956979711059], 6: [0.6549730996262775], 7: [0.8529673390049615], 8: [0.618033988749895], 9: [0.5257311121191344], 10: [0.48969702869403314], 11: [0.46417900254028893], 12: [1.0000000000000002], 13: [1.0000000000000002], 14: [1.0], 15: [1.000000000000001]} 

{0: [0.6666666666666667], 1: [0.8000000000000002], 2: [0.8571428571428575], 3: [0.8888888888888891], 4: [0.0], 5: [0.7151956979711059], 6: [0.6549730996262775], 7: [0.8529673390049615], 8: [0.618033988749895], 9: [0.5257311121191344], 10: [0.48969702869403314], 11: [0.46417900254028893], 12: [0.6666666666666667], 13: [0.6666666666666669], 14: [0.6666666666666666], 15: [0.6666666666666674]} 

{0: [0.6666666666666667], 1: [0.6000000000000001], 2: [0.7142857142857146], 3: [0.7777777777777779], 4: [0.0], 5: [0.7151956979711059], 6: [0.32748654981313874], 7: [0.6397255042537211], 8: [0.618033988749895], 

### Symmetries

In [19]:
# CALCULATING |AUT(G)| FOR EACH GRAPH

num_symmetries = []
generating_symmetries = []
for graph in graphs:
    adjacency_dict = graph.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=graph, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=graph, adjacency_dict=adjacency_dict)
    num_symmetries.append(num_automorphisms)
    generating_symmetries.append(generators)
with open("results/aut_G", 'wb') as file:
      pickle.dump((num_symmetries,generating_symmetries), file)

In [20]:
# CALCULATING |AUT(G')| FOR EACH GRAPH PERTURBATION

# One shadow node
num_symmetries_shadow1 = []
generating_symmetries_shadow1 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_shadow1.append(num_automorphisms)
    generating_symmetries_shadow1.append(generators)


# Two shadow nodes
num_symmetries_shadow2 = []
generating_symmetries_shadow2 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    G.add_node(N+1)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_shadow2.append(num_automorphisms)
    generating_symmetries_shadow2.append(generators)


# One pendent edge
num_symmetries_pendent = []
generating_symmetries_pendent = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    u = list(G.nodes())[N]
    v = list(G.nodes())[random.randint(0,N-2)]
    G.add_edge(u,v)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_pendent.append(num_automorphisms)
    generating_symmetries_pendent.append(generators)


# One deleted edge
idx = 0
num_symmetries_deleted = {}
generating_symmetries_deleted= {}
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G_ = P.Problems(p_type="custom", G=G)
    num_symmetries_deleted_vec = []
    generating_symmetries_deleted_vec = []
    for edge in G_.get_edges():
        G_reduced = G_.get_graph().copy()
        G_reduced.remove_edges_from([edge])
        instance = P.Problems(p_type="custom", G=G_reduced)
        adjacency_dict = instance.get_adjacency_dict()
        num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
        generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
        num_symmetries_deleted_vec.append(num_automorphisms)
        generating_symmetries_deleted_vec.append(generators)
    num_symmetries_deleted[idx] = num_symmetries_deleted_vec
    generating_symmetries_deleted[idx] = generating_symmetries_deleted_vec
    idx += 1

In [21]:
# CHECK

print(num_symmetries)
print(num_symmetries_shadow1)
print(num_symmetries_shadow2)
print(num_symmetries_pendent)
print(num_symmetries_deleted)

[24, 720, 40320, 3628800, 2, 2, 1, 1, 2, 2, 2, 4, 24, 12, 12, 20]
[24, 720, 40320, 3628800, 4, 2, 2, 1, 2, 2, 2, 4, 24, 12, 12, 20]
[48, 1440, 80640, 7257600, 12, 4, 6, 2, 4, 4, 4, 8, 48, 24, 24, 40]
[6, 120, 5040, 362880, 4, 1, 1, 1, 2, 2, 2, 2, 6, 2, 4, 2]
{0: [4, 4, 4, 4, 4, 4], 1: [48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48], 2: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 3: [80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640, 80640], 4: [4, 4], 5: [4, 1, 1, 4, 6, 1, 1, 4], 6: [2, 2, 2, 1, 2, 2, 1, 1, 4], 7: [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1], 8: [8, 2, 2], 9: